***Task***

Reddit comments: Data

10 words->sentence length

10 words+eos

preprocessing

one hot encoding and word2vec

Transformer(single head)

class

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Layer
from tensorflow.keras.layers import Embedding, Input, GlobalAveragePooling1D, Dense
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gensim
import gensim.downloader

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
def words(str):
  tokens=str.lower()
  tokens=str.split()
  str1=''

  for i in range(len(tokens)):
    if i==0:
      str1+=tokens[i]
    else:

      str1=str1+" "+tokens[i]
  return str1.lower()

In [ ]:
def cleaningPunc(str):
  str=re.sub(r'[^\w\s]','',str)
  return str

In [ ]:
def urlremove(sentence):
  sentence=re.sub(r'http\S+', '', sentence)
  sentence=re.sub(r'www\S+', '', sentence)
  return sentence


In [ ]:
def stopwordsRemove(str):
  str1=""
  stop_words=stopwords.words('english')
  str=str.split()

  for word in str:
    if not word in stop_words:
      str1+=word+' '
  return str1


In [ ]:
def cleaningdigits(str):
  str=re.sub(r'[\d+]','',str)
  return str

In [ ]:
def lemmatization(str):
  lemma=WordNetLemmatizer()
  str1=''
  tokens=str.split()
  for word in tokens:
    store=lemma.lemmatize(word)
    str1+=store+' '
  return str1.lower()


In [ ]:
url='https://drive.google.com/file/d/1xEyhcHjrjEo62k84kKu6mI9vttGtc-jV/view?usp=share_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
df=df.drop_duplicates()
df=df[df['Message'].notnull()]
df=df[df['Category'].notnull()]

label_encoder=preprocessing.LabelEncoder()
df['Category']=label_encoder.fit_transform(df['Category'])

texts=df['Message'].to_numpy()
labels=df['Category'].to_numpy()

x_train,x_test,y_train,y_test=train_test_split(texts,labels,test_size=0.3)


print(x_train.shape)




(3609,)


In [ ]:
for i in range(len(x_train)):
  sentence=x_train[i]

  sentence=words(sentence)

  sentence=cleaningPunc(sentence)

  sentence=stopwordsRemove(sentence)

  sentence=cleaningdigits(sentence)

  sentence=lemmatization(sentence)

  sentence=urlremove(sentence)
  x_train[i]=sentence


In [ ]:
for i in range(len(x_test)):
  sentence=x_test[i]

  sentence=words(sentence)

  sentence=cleaningPunc(sentence)

  sentence=stopwordsRemove(sentence)

  sentence=cleaningdigits(sentence)

  sentence=lemmatization(sentence)

  sentence=urlremove(sentence)
  x_test[i]=sentence

In [ ]:
#new_model= gensim.downloader.load('fasttext-wiki-news-subwords-300')
new_model=gensim.downloader.load('glove-twitter-25')
new_model.most_similar("bad")

[===============================================---] 95.4% 100.0/104.8MB downloaded


[('shit', 0.9544215202331543),
 ('crazy', 0.9532092213630676),
 ('but', 0.952229380607605),
 ('hell', 0.9521805047988892),
 ('right', 0.9486410617828369),
 ('like', 0.948320209980011),
 ('same', 0.9475184082984924),
 ('damn', 0.94697105884552),
 ('thing', 0.9445644617080688),
 ('way', 0.9423031210899353)]

In [ ]:
word='eating'
vec=new_model[word]
print(len(vec))



25


In [ ]:
token_index={}
counter=0
for sentence in x_train:
  tokens=sentence.split()
  for token in tokens:
    if not token in token_index:
      token_index.update({token:counter+1})
      counter=counter+1
print(len(token_index))


6431


In [ ]:
maxlen=len(token_index)
wv_size=len(vec)
results  = np.zeros(shape = (len(x_train),
                            maxlen,
                            wv_size))
print(results.shape)

results_test  = np.zeros(shape = (len(x_test),
                            maxlen,
                            wv_size))
print(results_test.shape)

(3609, 6431, 25)
(1548, 6431, 25)


In [ ]:
for i, documents in enumerate(x_train):
  for j, considered_word in list(enumerate(documents.split())):
    try:
      results[i, j, :] = new_model[considered_word]
    except:
      results[i,j,:]=np.zeros((wv_size))




In [ ]:
for i, documents in enumerate(x_test):
  for j, considered_word in list(enumerate(documents.split())):
    try:
      results[i, j, :] = new_model[considered_word]
    except:
      results[i,j,:]=np.zeros((wv_size))


In [ ]:
class TransformerBlock(Layer):
    def __init__(self, embed_dim, ff_dim, num_heads=1, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"),
             Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class TokenAndPositionEmbedding(Layer):
    def __init__(self):
        super(TokenAndPositionEmbedding, self).__init__()


    def call(self, x):
      document_len=len(x)
      n=10000
      #positions=np.zeros((document_len,maxlen,vocab_size))
      positions=np.zeros((maxlen,vocab_size))
      #for i in range(document_len):
      for j in range(maxlen):
        for k in np.arange(int(vocab_size/2)):
          denominator=np.power(n,2*k/vocab_size)
          positions[j,2*k]=np.sin(j/denominator)
          positions[j,2*k+1]=np.cos(j/denominator)

      return x + positions

In [ ]:
vocab_size=wv_size
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = Input(shape=(maxlen,vocab_size))

embedding_layer = TokenAndPositionEmbedding()
x = embedding_layer(inputs)
print(x.shape)
transformer_block = TransformerBlock(vocab_size, ff_dim)
x=transformer_block(x)
x=transformer_block(x)
x=transformer_block(x)
x=transformer_block(x)
x=transformer_block(x)
x=transformer_block(x)
#x=tf.keras.backend.mean(
#    x,
#    axis=1,
#    keepdims=False
#)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(20, activation="relu")(x)
x = Dropout(0.1)(x)
outputs = Dense(6, activation="softmax")(x)
print(outputs.shape)
model = Model(inputs=inputs, outputs=outputs)

(None, 6431, 25)
(None, 6)


In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

history = model.fit(results, y_train,
                    batch_size=128, epochs=32,
                    validation_data=(results_test, y_train)
                   )

Epoch 1/32


In [ ]:
results_calc = model.evaluate(results_test, y_test, verbose=2)

for name, value in zip(model.metrics_names, results_calc):
    print("%s: %.3f" % (name, value))